Used websites:

https://codelike.pro/create-a-crawler-with-rotating-ip-proxy-in-python/  
http://jonathansoma.com/lede/foundations-2017/classes/adv-scraping/advanced-scraping-form-submission/

In [1]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import random
import requests

In [2]:
# returns a request object from given url with random user agent and random proxy
def request(url:str,request_type='get',post_params={}) -> requests.models.Response:
    
    # check parameter requirements
    if request_type != 'get' and request_type != 'post':
        raise ValueError("request_type must be 'get' or 'post' of type str")
        
    if type(post_params) is not dict:
        raise ValueError("post_params must be of type dict()")
        
    if len(post_params) > 0:
        if request_type != 'post':
            raise ValueError("post_params can only be passed if request_type is set to 'post'")
    
    #  disable ANY annoying exception with a fallback to default user agent if anything goes wrong
    ua = UserAgent(fallback = 'Mozilla/5.0 (Windows; U; Windows NT 6.1; zh-HK) AppleWebKit/533.18.1 (KHTML, like Gecko) Version/5.0.2 Safari/533.18.5')

    # update saved database of user agents
    # fake_useragent stores collected data at os temp dir
    print('update user agent database')
    ua.update()
    
    print('get proxy list')
    # will contain proxies [ip, port]
    proxies = []

    # retrieve latest proxies from website
    proxies_req = requests.get(url='https://www.sslproxies.org/', headers={'user-agent': ua.random})
    proxies_req.encoding = 'utf-8'

    # create beatifulsoup instance, parse html and get section with proxy list of website
    parsed_html = BeautifulSoup(proxies_req.text, 'html.parser')
    proxies_table = parsed_html.find(id='proxylisttable')

    # save proxies from proxy list of website in the array
    for row in proxies_table.tbody.find_all('tr'):
        proxies.append({
        'ip':   row.find_all('td')[0].string,
        'port': row.find_all('td')[1].string
      })
        
    # retrieve a random index proxy
    def random_proxy_idx():
        return random.randint(0, len(proxies) - 1)
    
    print('try proxies')
    # try if proxy server is running, if not try other random proxy and user agent, try maximum x times then exit loop
    proxy_is_good = False
    i = 0
    x = 10
    while not proxy_is_good:
    
        # get dict for random proxy 
        random_proxy = proxies[random_proxy_idx()]
        random_proxy_dict = {'http': 'http://' + random_proxy['ip'] + ':' + random_proxy['port'],
                        'https': 'https://' + random_proxy['ip'] + ':' + random_proxy['port']}
    
        # try to get request object from url with random user agent and random proxy
        try:
            if request_type == 'get':
                req = requests.get(url=url, headers={'user-agent': ua.random},proxies=random_proxy_dict)
            elif request_type == 'post':
                req = requests.get(url=url, headers={'user-agent': ua.random},proxies=random_proxy_dict)
            else:
                raise ValueError("request_type must be 'get' or 'post' of type str")
                
        # if anything goes wrong try again with differnt user agent and proxy
        except:
            print('Error',random_proxy)
            if i == x:
                print('All',i,'proxies are not reachable')
                break
            i += 1
        # if everything goes right, end loop and return request object
        else:
            proxy_is_good = True
            print('Success',random_proxy)
    return req

In [3]:
req = request('https://www.daimler.com/karriere/jobsuche/', request_type='post', post_params={'j':2})

update user agent database


Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "C:\Users\Lenovo\Anaconda3\lib\site-packages\fake_useragent\utils.py", line 67, in get
    context=context,
  File "C:\Users\Lenovo\Anaconda3\lib\urllib\request.py", line 223, in urlopen
    return opener.open(url, data, timeout)
  File "C:\Users\Lenovo\Anaconda3\lib\urllib\request.py", line 526, in open
    response = self._open(req, data)
  File "C:\Users\Lenovo\Anaconda3\lib\urllib\request.py", line 544, in _open
    '_open', req)
  File "C:\Users\Lenovo\Anaconda3\lib\urllib\request.py", line 504, in _call_chain
    result = func(*args)
  File "C:\Users\Lenovo\Anaconda3\lib\urllib\request.py", line 1346, in http_open
    return self.do_open(http.client.HTTPConnection, req)
  File "C:\Users\Lenovo\Anaconda3\lib\urllib\request.py", line 1321, in do_open
    r = h.getresponse()
  File "C:\Users\Lenovo\Anaconda3\lib\http\client.py"

get proxy list
try proxies
Success {'ip': '193.238.98.206', 'port': '57437'}
